In [18]:
import numpy as np
from pprint import pprint

In [26]:
# Parse the game tree
def make_game_tree(src: str):
    with open(src) as fin:
        lines = fin.read().strip().split("\n")

    nodes = {}
    infosets = {}

    for line in lines:
        parts = line.split(" ")
        
        if parts[0] == "node":
            hist = parts[1]
            node_type = parts[2]

            if node_type == "player":
                # Looks like "/P1:r/ player 2 r p s"
                player = parts[3]
                actions = parts[5:]
                nodes[hist] = {
                    "type": "player",
                    "player": player,
                    "actions": tuple(actions),
                }

            else:
                assert node_type == "terminal"
                # Looks like "node /P1:r/P2:r/ terminal payoffs 1=0 2=0"
                payoffs = {}
                for payoff in parts[4:]:
                    p, v = payoff.split("=")
                    payoffs[p] = int(v)

                nodes[hist] = {
                    "type": "terminal",
                    "payoffs": payoffs
                }
        
        else:
            assert parts[0] == "infoset"
            # Looks like "infoset /P1:?/ nodes /P1:r/ /P1:p/ /P1:s/"
            hist = parts[1]
            info_nodes = parts[3:]

            actions = None
            player = None

            for info_hist in info_nodes:
                node = nodes[info_hist]
                assert node["type"] == "player"
                
                if actions is None:
                    actions = node["actions"]
                else:
                    # Must have same set of actions
                    assert node["actions"] == actions

                if player is None:
                    player = node["player"]
                else:
                    assert node["player"] == player

                infosets[hist] = {
                    "player": player,
                    "nodes": tuple(info_nodes),
                    "actions": actions,
                }

    return {
        "nodes": nodes,
        "infosets": infosets,
    }

game_tree = make_game_tree("./efgs/rpss.txt")
nodes = game_tree["nodes"]
infosets = game_tree["infosets"]

print("===== NODES =====")
pprint(nodes)
print()
print("===== INFOSETS =====")
pprint(infosets)

===== NODES =====
{'/': {'actions': ('r', 'p', 's'), 'player': '1', 'type': 'player'},
 '/P1:p/': {'actions': ('r', 'p', 's'), 'player': '2', 'type': 'player'},
 '/P1:p/P2:p/': {'payoffs': {'1': 0, '2': 0}, 'type': 'terminal'},
 '/P1:p/P2:r/': {'payoffs': {'1': 1, '2': -1}, 'type': 'terminal'},
 '/P1:p/P2:s/': {'payoffs': {'1': -2, '2': 2}, 'type': 'terminal'},
 '/P1:r/': {'actions': ('r', 'p', 's'), 'player': '2', 'type': 'player'},
 '/P1:r/P2:p/': {'payoffs': {'1': -1, '2': 1}, 'type': 'terminal'},
 '/P1:r/P2:r/': {'payoffs': {'1': 0, '2': 0}, 'type': 'terminal'},
 '/P1:r/P2:s/': {'payoffs': {'1': 1, '2': -1}, 'type': 'terminal'},
 '/P1:s/': {'actions': ('r', 'p', 's'), 'player': '2', 'type': 'player'},
 '/P1:s/P2:p/': {'payoffs': {'1': 2, '2': -2}, 'type': 'terminal'},
 '/P1:s/P2:r/': {'payoffs': {'1': -1, '2': 1}, 'type': 'terminal'},
 '/P1:s/P2:s/': {'payoffs': {'1': 0, '2': 0}, 'type': 'terminal'}}

===== INFOSETS =====
{'/': {'actions': ('r', 'p', 's'), 'nodes': ('/',), 'player'

In [27]:
# Extract a tree-form decision process (TFDP) from the game tree
# Set of decision nodes j in J_i of TFDP coincides with set of their infosets
# Set of actions at J_i coincides with set of actions at any node in J_i
# At each observation node k in K, player observes signal s in S_k

# A pair (j, a) of (decision node, action) constitutes a "non-empty sequence."
# Sigma: set of sequences. Strategies real-valued vectors over Sigma.

In [ ]:
# Generate a TFDP from the game tree
def get_sequences(infosets: dict, player: str):
    """
    Deterministically generate a list of decision node/action pairs (sequences)
        given the game tree infosets
    """
    seqs = []

    for info_hist, info_node in sorted(infosets.items()):
        if info_node["player"] != player:
            continue

        for action in info_node["actions"]:
            seqs.append((info_hist, action))

    return seqs

print(get_sequences(infosets, "1"))

[('/', 'r'), ('/', 'p'), ('/', 's')]


In [ ]:
# How to represent a strategy?
# For each (j, a), list a real number
# Probabilities must add up